<a href="https://colab.research.google.com/github/akilaIduwara/SDGP_Project/blob/Akila_Induwara/SDGP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import Required Libraries
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
#Load and Preprocess Data

# Load data (replace with your actual file path)
df = pd.read_csv('JWD_L8_DD_ERTmodel.csv', skiprows=1, header=0)

# Clean invalid resistivity values
df = df[df['Log10Res'] != -9999]

# Convert Log10Res to resistivity (ohm-m)
df['Resistivity'] = 10 ** df['Log10Res']

# Create target variable: 1 if resistivity is between 20 and 100 ohm-m
df['is_water'] = np.where((df['Resistivity'] >= 20) & (df['Resistivity'] <= 100), 1, 0)

# Extract features and target
features = df[['Dist', 'X_NAD83UTMz16N', 'Y_NAD83UTMz16N', 'Elev']]
target = df['is_water']

In [ ]:
#Train XGBoost Model

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=42
)

# Initialize and train XGBoost classifier
model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False
)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


In [ ]:
#Identify Water Resources and Display Results

# Predict water locations in the entire dataset
df['predicted_water'] = model.predict(features)

# Filter rows where water is predicted
water_locations = df[df['predicted_water'] == 1]

# Display results (Depth = absolute value of Elev)
results = water_locations[[
    'X_NAD83UTMz16N', 'Y_NAD83UTMz16N', 'Elev'
]].rename(columns={'Elev': 'Depth'})

results['Depth'] = np.abs(results['Depth'])  # Convert elevation to depth

print("\nIdentified Water Resources:")
print(results.to_string(index=False))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')